# Gráficos de Círculo, Caja, Dispersión y Burbuja

## Objectivos

Después de completar este laboratorio será capas de:

*   Explorar más la biblioteca de Matplotlib
*   Crear gráficos de círculo, caja, dispersión y burbuja


## Tabla de Contenidos

1.  [Explorando Conjunto de Datos con *pandas* y *Matplotlib*](#0)
2.  [Descarga y Preparación de Datos](#2)
3.  [Visualización de datos usando Matplotlib](#4)
4.  [Gráfico de Circulo](#6)
5.  [Gráfico de Caja](#8)
6.  [Gráfico de Dispersión](#10)
7.  [Gráfico de Burbuja](#12)


# Explorando Conjunto de Datos con *pandas* y *Matplotlib*<a id="0"></a>

Kit de herramientas: El curso se basa en gran medida en [**pandas**](http://pandas.pydata.org) y [**Numpy**](http://www.numpy.org/) para la manipulación, el análisis y la visualización de datos. La biblioteca principal de visualización que se está explorando en el curso es [**Matplotlib**](http://matplotlib.org/).

Conjunto de datos: Inmigración a Canada desde 1980 a 2013 - [International migration flows to and from selected countries - The 2015 revision](http://www.un.org/en/development/desa/population/migration/data/empirical2/migrationflows.asp) desde el sitio ewb de las Naciones Unidas.

El conjunto de datos contiene datos anuales sobre los flujos de migrantes internacionales registrados por los países de destino. Los datos presentan tanto las entradas como las salidas según el lugar de nacimiento, ciudadanía o lugar de residencia anterior/próxima tanto para extranjeros como para nacionales. Este laboratorio se concentra en los datos de inmigración canadiense.

# Descarga y Preparación de Datos <a id="2"></a>


Lo primero que hay que realizar es la instalación de **openpyxl** (antiguamente conocido como **xlrd**) que es un módulo que **pandas** requiere para leer archivos Excel.

In [ ]:
!pip install openpyxl

A continuación, se procederá a cargar los dos módulos claves para hacer análisis de datos **pandas** y
 **numpy**.

In [ ]:
import numpy as np  # util para computación científica en Python
import pandas as pd # biblioteca que contienen la estructura de datos de uso principal

Ahora, se carga el conjunto de datos con los datos de la inmigración en Canadá usando el método de **pandas** `read_excel()`.

In [ ]:
df_can = pd.read_excel(
    'datos/Canada.xlsx',
    sheet_name='Canada by Citizenship',
    skiprows=range(20),
    skipfooter=2)

print('Datos leídos y cargados en un dataframe de pandas!')

Mostrar las primeras 5 filas del conjunto de datos usando la función `head()`.


In [ ]:
df_can.head()

Descubrir cuantas entradas hay en el conjunto de datos.


In [ ]:
# dimensión del dataframe (filas, columnas)
df_can.shape  

Limpiar datos. Se realizarán algunas modificaciones al conjunto de datos original para que sea más fácil crear visualizaciones. La explicación de estas modificaciones están detalladas en los laboratorios anteriores *"Introducción a Matplotlib y Gráficos de línea"* y *"Gráficos de Área, Barras e Histogramas"*.


In [ ]:
# limpiar el conjunto de datos para eliminar columnas innecesarias (por ejemplo, REG) 
df_can.drop(['AREA', 'REG', 'DEV', 'Type', 'Coverage'], axis=1, inplace=True)

# cambiar el nombre de algunas de las columnas para que tengan sentido
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)

# para mantener la coherencia, asegúrese de que todas las etiquetas de las columnas sean de tipo cadena de caracteres o string
df_can.columns = list(map(str, df_can.columns))

# establecer el nombre del país como índice. Esto es útil para buscar rápidamente países utilizando el método .loc
df_can.set_index('Country', inplace=True)

# agregar la columna total
df_can['Total'] = df_can.sum(axis=1, numeric_only=True)

# finalmente, se crea una lista de años desde 1980 - 2013, esto será útil cuando comencemos a graficar los datos
years = list(map(str, range(1980, 2014)))
print('Dimensión de los datos:', df_can.shape)

# Visualización de datos usando Matplotlib<a id="4"></a>


Importar la biblioteca `matplotlib`.


In [ ]:
# se hará uso del despliegue de figuras en línea o dentro de Jupyter Notebook
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot')  # opcional: para estilo similar a ggplot

# verificar la versión de Matplotlib
print('Matplotlib version: ', mpl.__version__)

# Gráfico de Círculo <a id="6"></a>

Un *gráfico de círculo* es un gráfico que muestra proporciones numéricas al dividir un círculo (o un pastel) en porciones proporcionales. Estos gráficos se utilizan ampliamente en los negocios y los medios. Podemos crear gráficos de círculos en Matplotlib pasando la palabra clave `kind=pie`.

Se usará un gráfico de círculo para explorar la proporción (porcentaje) de nuevos inmigrantes agrupados por continentes durante todo el período de 1980 a 2013.


Paso 1: Reunir los datos.

Se usará el método `groupby` de *pandas* para resumir los datos de inmigración por `Continent`. El proceso general de `groupby` implica los siguientes pasos:

1. **Dividir:** Dividir los datos en grupos según algunos criterios.
2. **Aplicar:** Aplicar una función a cada grupo de forma independiente:
     .sum()
     .count()
     .mean()
     .std()
     .aggregate()
     .apply()
     .etc..
3. **Combinar:** combinar los resultados en una estructura de datos.


![groupby](imagenes/groupby.png)


In [ ]:
# agrupar países por continente y aplicar la función sum() 
df_continentes = df_can.groupby('Continent', axis=0).sum()

# la salida del método groupby es un objeto 'groupby'. 
# y esta no se puede usar hasta que se le aplique una función (por ejemplo, .sum())
print(type(df_can.groupby('Continent', axis=0)))

df_continentes.head()

Paso 2: Graficar los datos. Se debe pasar la palabra clave `kind = 'pie'`, junto con los siguientes parámetros adicionales:

*   `autopct` -  es una cadena o función utilizada para etiquetar las porciones con su valor numérico. La etiqueta se colocará dentro de la porción. Si es una cadena de formato, la etiqueta será `fmt%pct`.
*   `startangle` - gira el inicio del gráfico en grados de ángulo en sentido contrario a las agujas del reloj desde el eje x.
*   `shadow` - Dibuja una sombra debajo del pastel (para dar una sensación de 3D).


In [ ]:
# autopct crear %, el ángulo de inicio representa el punto de inicio
df_continentes['Total'].plot(kind='pie',
                            figsize=(5, 6),
                            autopct='%1.1f%%', # añadir en porcentajes
                            startangle=90,     # ángulo de inicio 90° (África)
                            shadow=True,       # añadir sombra     
                            )

plt.title('Inmigración a Canadá por Continente [1980 - 2013]')
plt.axis('equal') # Establece el gráfico para que parezca un círculo.

plt.show()

La imagen anterior no es muy clara, los números y el texto se superponen en algunos casos. Se deben hacer algunas modificaciones para mejorar la imagen:

* Eliminar las etiquetas de texto en el gráfico pasando `legend` y agregar una leyenda separada usando `plt.legend()`.
* Mover los porcentajes para que queden fuera del gráfico pasando el parámetro `pctdistance`.
* Definir un conjunto personalizado de colores para los continentes pasando el parámetro `colors`.
* **Explotar** el gráfico para enfatizar los tres continentes más bajos (África, América del Norte y América Latina y el Caribe) pasando el parámetro `explode`.


In [ ]:
lista_colores = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightgreen', 'pink']
lista_explotar = [0.1, 0, 0, 0, 0.1, 0.1] # relación para cada continente del inclinado de cada porción.
df_continentes['Total'].plot(kind='pie',
                            figsize=(15, 6),
                            autopct='%1.1f%%', 
                            startangle=90,    
                            shadow=True,       
                            labels=None,           # desactiva las etiquetas en el gráfico
                            pctdistance=1.12,      # relación entre el centro de cada sector circular y el inicio del texto generado por autopct 
                            colors=lista_colores,  # añadir colores personalizados
                            explode=lista_explotar # explotar los 3 continentes más bajos
                            )

# aumentar el título en un 12% para que coincida con pctdistance
plt.title('Inmigración a Canada por Continente [1980 - 2013]', y=1.12) 

plt.axis('equal') 

# añadir leyenda
plt.legend(labels=df_continentes.index, loc='upper left') 

plt.show()

**Pregunta:** Usando un gráfico de círculo, explore la proporción (porcentaje) de nuevos inmigrantes agrupados por continentes en el año 2013.

**Nota**: puede que se deba jugar con los valores de exploración para corregir los valores de división superpuestos.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo



Doble click **aquí** para ver la solución

<!--
#La respuesta correcta es:

lista_explotar = [0.0, 0, 0, 0.1, 0.1, 0.2] # relación para cada continente del inclinado de cada porción.

df_continentes['2013'].plot(kind='pie',
                            figsize=(15, 6),
                            autopct='%1.1f%%', 
                            startangle=90,    
                            shadow=True,       
                            labels=None,           # desactiva las etiquetas en el gráfico
                            pctdistance=1.12,      # trelación entre el centro de cada sector circular y el inicio del texto
                            explode=lista_explotar # 'explotar' los 3 continentes más bajos
                            )

# aumentar el título en un 12% para que coincida con pctdistance
plt.title('Inmigración a Canadá por Continente en 2013', y=1.12) 
plt.axis('equal') 

# añadir leyenda
plt.legend(labels=df_continentes.index, loc='upper left') 

# Mostrar gráfico
plt.show()

-->    

# Gráfico de caja <a id="8"></a>

Un `gráfico de caja` es una forma de representar estadísticamente la *distribución* de los datos a través de cinco dimensiones principales:

* **Mínimo:** El número más pequeño en el conjunto de datos excluyendo los valores atípicos.
* **Primer cuartil:** Número medio entre el `mínimo` y la `mediana`.
* **Segundo cuartil (mediana):** Número medio del conjunto de datos (ordenado).
* **Tercer cuartil:** Número medio entre `mediana` y `máximo`.
* **Máximo:** El número más grande en el conjunto de datos excluyendo los valores atípicos.

![caja](imagenes/caja.png)


Para hacer un gráfico de caja o `boxplot`, se debe usar `kind=box` en el método `plot` invocado en una serie *pandas* o dataframe.

Generar un gráfico de caja para los inmigrantes japoneses entre 1980 y 2013.


Paso 1: Obtener el subconjunto del conjunto de datos. Si bien se están extrayendo los datos de un solo país, estos se obtendrán como un dataframe. Así, se llamará al método `dataframe.describe()` para ver los percentiles.

In [ ]:
# para obtener el dataframe, colocar corchetes adicionales alrededor de 'Japón'.
df_japon = df_can.loc[['Japan'], years].transpose()
df_japon.head()

Paso 2: Graficar configurando `kind='box'`.


In [ ]:
df_japon.plot(kind='box', figsize=(8, 6))

plt.title('Gráfico de Caja de Inmigrantes Japoneses entre 1980 - 2013')
plt.ylabel('Número de Inmigrantes')

plt.show()

Algunas observaciones claves de la gráfica anterior:

1. El número mínimo de inmigrantes es de alrededor de 200 (min), el número máximo es de alrededor de 1300 (max) y el número medio de inmigrantes es de alrededor de 900 (mediana).
2. El 25% de los años del período 1980 - 2013 tuvo un recuento anual de inmigrantes de \~500 o menos (primer cuartil).
3. El 75% de los años del período 1980 - 2013 tuvo un recuento anual de inmigrantes de \~1100 o menos (tercer cuartil).

Podemos ver los números reales llamando al método `describe()` en el dataframe.


In [ ]:
df_japon.describe()

Uno de las ventajas de los gráficos de caja es comparar la distribución de múltiples conjuntos de datos. En uno de los laboratorios anteriores, se observó que China e India tenían tendencias de inmigración similares. Analizar más a fondo estos dos países usando diagramas de caja.

**Pregunta:** Compare la distribución del número de nuevos inmigrantes de India y China para el período 1980 - 2013.


Paso 1: Obtener el conjunto de datos para China e India y llamar al dataframe **df_CI**.



In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo




Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

df_CI= df_can.loc[['China', 'India'], years].transpose()
df_CI.head()

-->    

Observar los percentiles asociados a ambos países utilizando el método `describe()`.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo


Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

df_CI.describe()

-->    

Paso 2: Graficar los datos.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo


Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

df_CI.plot(kind='box', figsize=(10, 7))

plt.title('Gráfico de Caja de Inmigrantes desde China e India entre 1980 - 2013')
plt.ylabel('Número de Inmigrantes')

plt.show()

--> 

Se puede observar que, si bien ambos países tienen aproximadamente la misma población inmigrante mediana (\~20.000), el rango de población inmigrante de China está más disperso que el de India. La población máxima de India para cualquier año (36.210) es alrededor de un 15% más baja que la población máxima de China (42.584).


Si se prefiere crear diagramas de caja horizontales, puede pasar el parámetro `vert` en la función **plot** y asignarle el valor *False*. También puede especificar un color diferente del color rojo predeterminado.


In [ ]:
# Gráfico de Caja horizontal
 
df_CI.plot(kind='box', figsize=(10, 7), color='blue', vert=False)

plt.title('Gráfico de Caja de Inmigrantes desde China e India entre 1980 - 2013')
plt.xlabel('Número de Inmigrantes')

plt.show()

**Subgráficos**

En ocasiones se requiere mostrar varios gráficos dentro de la misma figura. Por ejemplo, se podría realizar una comparación lado a lado del diagrama de caja con el diagrama de línea de la inmigración de China e India.

Para mostrar varios gráficos juntos, se puede crear una **`figura`** (lienzo general) y dividirla en subgráficos o **`subplot`**, cada uno de las cuales contiene un gráfico. Con **subpplots**, generalmente se trabaja con la **capa de artista** en lugar de la **capa de secuencias de comandos**.

La sintaxis típica es:

```python
    fig = plt.figure() # crea una figura
    ax = fig.add_subplot(nrows, ncols, plot_number) # crea subgráficos
```

Donde

*   `nrows` y `ncols` se utilizan para dividir la figura en (`nrows` \* `ncols`) subejes,
*   `plot_number` se usa para identificar el subgráfico particular que esta función creará dentro de la cuadrícula. `plot_number` comienza en 1, se incrementa en las filas primero y tiene un máximo de `nrows` \* `ncols` como se muestra a continuación.

![caja](imagenes/subgrafico.png)

Luego podemos especificar en qué subgráfico colocar cada gráfico pasando el parámetro `ax` en el método `plot()` de la siguiente manera:


In [ ]:
fig = plt.figure() # crear figura

ax0 = fig.add_subplot(1, 2, 1) # agregar subgráfico 1 (1 fila, 2 columnas, primer gráfico)
ax1 = fig.add_subplot(1, 2, 2) # agregar subgráfico 2 (1 fila, 2 columnas, segundo gráfico). Ver consejo bajo

# Subgráfico 1: Gráfico de caja
df_CI.plot(kind='box', color='blue', vert=False, figsize=(20, 6), ax=ax0) # agregar a subgráfico 1
ax0.set_title('Gráfico de Caja de Inmigrantes desde China e India (1980 - 2013)')
ax0.set_xlabel('Número de Inmigrantes')
ax0.set_ylabel('Países')

# Subgráfico 2: Gráfico de línea
df_CI.plot(kind='line', figsize=(20, 6), ax=ax1) # agregar a subgráfico 2
ax1.set_title ('Gráfico de Línea de Inmigrantes desde China e India (1980 - 2013)')
ax1.set_ylabel('Número de Inmigrantes')
ax1.set_xlabel('Años')

plt.show()

**Consejo sobre la convención de los subgráficos**

En el caso de que `nrows`, `ncols` y `plot_number` sean todos menores que 10, existe la conveniencia de que en su lugar se pueda dar un número de 3 dígitos, donde las centenas representan `nrows`, las decenas representan `ncols ` y las unidades representan `plot_number`. Por ejemplo,

```python
    subplot(211) == subplot(2, 1, 1)
```

produce subejes en una figura que representa el gráfico superior (es decir, la primera) en una cuadrícula compuesta de 2 filas por 1 columna (en realidad no existe una cuadrícula, pero conceptualmente así es como se ha posicionado el subgráfico generado).

Ahora se podría probar algo un poco más avanzado.

Anteriormente, identificamos los 15 principales países en función de la inmigración total de 1980 a 2013.

**Pregunta:** Crear un gráfico de caja para visualizar la distribución de los 15 principales países (según la inmigración total) agrupados por las *décadas* `1980`, `1990` y `2000`.

Paso 1: Obtener el conjunto de datos. Obtenga los 15 principales países según la población inmigrante total. Nombre el marco de datos **df_top15**.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo

# df_top15 = ...
# df_top15


Doble click **aquí** para ver la solución

<!--
#La respuesta correcta es:  

df_top15 = df_can.sort_values(['Total'], ascending=False, axis=0).head(15)
df_top15

-->    

Paso 2: crear un nuevo dataframe que contenga el total de cada década. Una forma de hacer eso:

1. Crear una lista de todos los años en las décadas 80, 90 y 00.
2. Cortar el dataframe original `df_can` para crear una serie de cada década y sumar sus años para cada país.
3. Combinar las tres series en un nuevo dataframe. Llamar al dataframe **df_nuevo**.

In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo


Doble click **aquí** para ver la solución

<!--
#La respuesta correcta es: 
    
# crear una lista de todos los años en las décadas 80, 90 y 00
años_80s = list(map(str, range(1980, 1990))) 
años_90s = list(map(str, range(1990, 2000))) 
años_00s = list(map(str, range(2000, 2010))) 

# corte el dataframe original df_can para crear una serie para cada década
df_80s = df_top15.loc[:, años_80s].sum(axis=1) 
df_90s = df_top15.loc[:, años_90s].sum(axis=1) 
df_00s = df_top15.loc[:, años_00s].sum(axis=1)

# combinar las tres series en un nuevo dataframe
df_nuevo = pd.DataFrame({'1980s': df_80s, '1990s': df_90s, '2000s':df_00s}) 

# mostrar el dataframe
df_nuevo.head()

--> 


Conocer más sobre las estadísticas asociadas al dataframe usando el método `describe()`.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo


Doble click **aquí** para ver la solución

<!--
#La respuesta correcta es:

df_nuevo.describe()

-->    

Paso 3: Generar los gráficos de Caja.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo



Doble click **aquí** para ver la solución

<!--
#La respuesta correcta es:

df_nuevo.plot(kind='box', figsize=(10, 6))

plt.title('Inmigración de los 15 países top para las décadas 80s, 90s y 2000s')

plt.show()

-->    

Observar cómo el gráfico de caja difiere de la tabla de resumen creada. El gráfico de caja escanea los datos e identifica los valores atípicos. Para ser un valor atípico, el valor de los datos debe ser:

* mayor que Q3 en al menos 1,5 veces el rango intercuartíl (IQR), o,
* menor que Q1 en al menos 1,5 veces el IQR.

Ver la década de 2000 como ejemplo: 

* Q1 (25%) = 36.101,5
* Q3 (75%) = 105.505,5
* IQR = Q3 - Q1 = 69.404

Utilizando la definición de valor atípico, cualquier valor que sea mayor que Q3 en 1,5 veces IQR se marcará como valor atípico.

Valor atípico > 105.505,5 + (1,5 \* 69.404)
Valor atípico > 209.611,5

In [ ]:
# Comprobar cuántas entradas están por encima del umbral de valores atípicos
# para la década del 2000s
#

df_prueba=df_nuevo.reset_index()
print(df_prueba)
print(df_prueba[df_prueba['2000s']> 209611.5][['Country','2000s']])


datos_estadisticos=df_nuevo.describe()
valor_atipico=[]
valor_atipico_lista=[]
for i, decada in enumerate(datos_estadisticos.columns):
  valor_atipico.append(datos_estadisticos.loc['75%'][decada] + (datos_estadisticos.loc['75%'][decada]-datos_estadisticos.loc['25%'][decada])*1.5)
  valor_atipico_lista.append(df_prueba[df_prueba[decada]>valor_atipico[i]][['Country',decada]])
valor_atipico_lista


Doble click **aquí** para ver la solución

<!--
#La respuesta correcta es:

df_prueba=df_nuevo.reset_index()
print(df_prueba)
print(df_prueba[df_prueba['2000s']> 209611.5][['Country','2000s']])


datos_estadisticos=df_nuevo.describe()
valor_atipico=[]
valor_atipico_lista=[]
for i, decada in enumerate(datos_estadisticos.columns):
  valor_atipico.append(datos_estadisticos.loc['75%'][decada] + (datos_estadisticos.loc['75%'][decada]-datos_estadisticos.loc['25%'][decada])*1.5)
  valor_atipico_lista.append(df_prueba[df_prueba[decada]>valor_atipico[i]][['Country',decada]])
valor_atipico_lista

-->    

<!-- The correct answer is:
new_df[new_df['2000s']> 209611.5]
-->


China e India se consideran valores atípicos ya que su población en la década supera los 209.611,5.

El diagrama de caja es una herramienta de visualización avanzada y hay muchas opciones y personalizaciones que exceden el alcance de esta práctica de laboratorio. Por favor refiérase a [Documentación de Matplotlib](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html?highlight=boxplot#matplotlib.pyplot.boxplot) para más información acerca de los gráficos de caja.


# Gráficos de Dispersión <a id="10"></a>

Un *gráfico de dispersión* es útil para comparar variables entre sí. Los gráficos de *dispersión* se parecen a los *gráficos de líneas* en el sentido de que mapean variables independientes y dependientes en un gráfico de 2 dimensiones. Si bien los puntos de datos están conectados entre sí por una línea en un gráfico de líneas, no están conectados en un gráfico de dispersión. Se considera que los datos en un diagrama de dispersión expresan una tendencia. Con un análisis más profundo usando herramientas como la regresión, podemos calcular matemáticamente esta relación y usarla para predecir tendencias fuera del conjunto de datos.

Para entender su funcionamiento se va a trabajar en lo siguiente:

Usando un *gráfico de dispersión*, visualizar la tendencia de la inmigración total a Canadá (todos los países combinados) para los años 1980 - 2013.


Paso 1: Obtener el conjunto de datos. Como se requiere usar la relación entre `años` y `total de población`, se convertirán los `años` al tipo `int`.


In [ ]:
# usar el método sum() para obtener la población total por año
df_total = pd.DataFrame(df_can[years].sum(axis=0))

# cambiar los años al tipo entero (útil para la regresión más adelante)
df_total.index = map(int, df_total.index)

# restablecer el índice para volver a colocarlo como una columna en el dataframe df_total
df_total.reset_index(inplace = True)

# renombrar columnas
df_total.columns = ['year', 'total']

# mostrar el dataframe resultante
df_total.head()

Paso 2: Graficar los datos. En `Matplotlib`, se puede crear un conjunto de gráficos `scatter` configurando `kind='scatter'` como argumento del tipo de gráfico. También se  necesita pasar las palabras clave `x` e `y` para especificar las columnas que van en el eje x e y.

In [ ]:
df_total.plot(kind='scatter', x='year', y='total', figsize=(10, 6), color='darkblue')

plt.title('Inmigración Total a Canadá entre 1980 - 2013')
plt.xlabel('Años')
plt.ylabel('Número de Inmigrantes')

plt.show()

Notar cómo el diagrama de dispersión no conecta los puntos de datos entre sí. Se puede observar una tendencia al alza en los datos: a medida que pasan los años, aumenta el número total de inmigrantes. También, se puede analizar matemáticamente esta tendencia al alza utilizando una línea de regresión (línea de mejor ajuste).


Para trazar una línea de mejor ajuste y usarla para predecir el número de inmigrantes en 2015 se deben seguir los siguientes pasos.

Paso 1: Obtener la ecuación de la línea de mejor ajuste. Para esto se usará el método `polyfit()` de **Numpy** pasando los siguientes parámetros:

* `x`: coordenadas x de los datos.
* `y`: coordenadas y de los datos.
* `deg`: Grado de ajuste del polinomio. 1 = lineal, 2 = cuadrático, y así sucesivamente.


In [ ]:
x = df_total['year']      # año en el eje x
y = df_total['total']     # total en el eje y
ajuste = np.polyfit(x, y, deg=1)

ajuste

La salida es un arreglo con los coeficientes polinómicos, las potencias más altas primero. Dado que estamos trazando una regresión lineal `y= a * x + b`, nuestra salida tiene 2 elementos `[5.56709228e+03, -1.09261952e+07]` con la pendiente en la posición 0 y la intersección en la posición 1.

Paso 2: Trazar la línea de regresión en el *diagrama de dispersión*.


In [ ]:
df_total.plot(kind='scatter', x='year', y='total', figsize=(10, 6), color='darkblue')

plt.title('Inmigración Total a Canadá entre 1980 - 2013')
plt.xlabel('Años')
plt.ylabel('Número de Inmigrantes')

# trazar la línea del mejor ajuste
plt.plot(x, ajuste[0] * x + ajuste[1], color='red') # recordar que x son los años
plt.annotate('y={0:.0f} x + {1:.0f}'.format(ajuste[0], ajuste[1]), xy=(2000, 150000))

plt.show()

# imprimir la línea de mejor ajuste
'Número de Inmigrantes = {0:.0f} * Año + {1:.0f}'.format(ajuste[0], ajuste[1]) 

Al usar la ecuación de recta de mejor ajuste, se puede estimar el número de inmigrantes en 2015:

```python
Nº Inmigrantes = 5567 * Año - 10926195
Nº Inmigrantes = 5567 * 2015 - 10926195
Nº Inmigrantes = 291.310
```

En comparación con el informe anual de 2016 de Citizenship and Immigration Canada (CIC) (https://www.canada.ca/en/immigration-refugees-citizenship/corporate/publications-manuals/annual-report-parliament-immigration-2016.html), se puede ver observar que Canadá aceptó 271.845 inmigrantes en 2015, que es bastante bueno. El valor estimado de 291,310 está dentro del 7% del número real, lo cual es bastante bueno considerando que nuestros datos originales provienen de las Naciones Unidas (y pueden diferir ligeramente de los datos de CIC).


**Pregunta**: .Crear un gráfico de dispersión de la inmigración total de Dinamarca, Noruega y Suecia a Canadá desde 1980 hasta 2013.


**Paso 1**: Obtener los datos:

1. Crear un dataframe que consista únicamente en los números asociados con Dinamarca, Noruega y Suecia. Nombrarlo **df_paises**.
2. Sumar las cifras de inmigración de los tres países para cada año y convierta el resultado en un marco de datos. Nombrar este nuevo marco de datos **df_totales**.
3. Restablezca el índice en su lugar.
4. Cambie el nombre de las columnas a **año** y **totales**.
5. Muestre el dataframe resultante.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo




Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

# crear dataframe df_paises
df_paises = df_can.loc[['Denmark', 'Norway', 'Sweden'], years].transpose()

# crear df_totales sumando los años para los tres países
df_totales = pd.DataFrame(df_paises.sum(axis=1))

# restablecer el índice para volver a colocarlo como una columna en el dataframe
df_totales.reset_index(inplace=True)

# renombrar las columnas
df_totales.columns = ['año', 'totales']

# cambiar la columna año de string a int para crear el gráfico de dispersión
df_totales['año'] = df_totales['año'].astype(int)

# mostrar el dataframe final
df_totales.head()

-->

<!--

#La respuesta correcta es:  

# crear dataframe df_paises
df_paises = df_can.loc[['Denmark', 'Norway', 'Sweden'], years].transpose()

# crear df_totales sumando los años para los tres países
df_totales = pd.DataFrame(df_paises.sum(axis=1))

# restablecer el índice para volver a colocarlo como una columna en el dataframe
df_totales.reset_index(inplace=True)

# renombrar las columnas
df_totales.columns = ['año', 'totales']

# cambiar la clumna año de string a int para crear el gráfico de dispersión
df_totales['año'] = df_total['año'].astype(int)

# mostrar el dataframe final
df_totales.head()

-->

**Paso 2**: Generar el gráfico de dispersión representando el total frente al año en **df_totales**.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo



Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

# crear el gráfico de dispersión
df_totales.plot(kind='scatter', x='año', y='totales', figsize=(10, 6), color='darkblue')

# agregar el título y etiquetas de los ejes
plt.title('Inmigración desde Dinamarca, Noruega, y Suecia a Canadá entre 1980 - 2013')
plt.xlabel('Año')
plt.ylabel('Número de Inmigrantes')

# mostrar gráfico
plt.show()

-->

# Gráfico de Burbuja <a id="12"></a>

Un *diagrama de burbujas* es una variación del *diagrama de dispersión* que muestra tres dimensiones de datos (x, y, z). Los puntos de datos se reemplazan con burbujas, y el tamaño de la burbuja está determinado por la tercera variable `z`, también conocida como peso. En `maplotlib`, podemos pasar un arreglo o escalar al parámetro `s` al método `plot()`, que contiene el peso de cada punto.

**Analizar el efecto de la gran depresión argentina**.

Argentina sufrió una gran depresión entre 1998 a 2002, que provocó un desempleo generalizado, disturbios, la caída del gobierno y el incumplimiento de pago de la deuda externa del país. En términos de ingresos, más del 50% de los argentinos eran pobres, y siete de cada diez niños Argentinos eran pobres en la profundidad de la crisis de 2002.

Se podría analizar el efecto de esta crisis y comparar la inmigración Argentina con la de su vecino Brasil. Esta comparación se puede realizar usando un *diagrama de burbuja* de la inmigración de Brasil y Argentina para los años 1980 - 2013. Se establecerán los pesos para la burbuja como el valor *normalizado* de la población para cada año.

**Paso 1**: Obtener los datos de Brasil y Argentina. Como en el ejemplo anterior, se convertirá `Years` al tipo entero y se incluirá en el dataframe.


In [ ]:
# transpuesta del dataframe
df_can_t = df_can[years].transpose()

# convertir el índice de los años al tipo entero
df_can_t.index = map(int, df_can_t.index)

# etiquetar el índice. Este será automáticamente el nombre de la columna cuando restablezcamos el índice
df_can_t.index.name = 'Año'

# restablecer el índice para pasar el año como una columna
df_can_t.reset_index(inplace=True)

# visualizar los cambios
df_can_t.head()

**Paso 2**: Crear los pesos normalizados.

Existen varios métodos de normalización en estadística, cada uno con su propio uso. En este caso, utilizaremos [escalado de variables](https://en.wikipedia.org/wiki/Feature_scaling) para llevar todos los valores al rango \[0, 1]. La formula general es:

$$X'=\frac{X-X_{min}}{X_{max}-X_{min}} $$

donde $X$ es el valor original, $X'$ es el valor normalizado correspondiente. La fórmula establece el valor máximo en el conjunto de datos en 1 y establece el valor mínimo en 0. El resto de los puntos de datos se escalan a un valor entre 0-1 según corresponda.

In [ ]:
# normalizar los datos de Brazil
norm_brazil = (df_can_t['Brazil'] - df_can_t['Brazil'].min()) / (df_can_t['Brazil'].max() - df_can_t['Brazil'].min())

# normalizar los datos de Argentina
norm_argentina = (df_can_t['Argentina'] - df_can_t['Argentina'].min()) / (df_can_t['Argentina'].max() - df_can_t['Argentina'].min())


**Paso 3**: Graficar los datos.

* Para trazar dos gráficos de dispersión diferentes en un gráfico, podemos incluir los ejes de un gráfico en el otro pasándolo a través del parámetro `ax`.
* También pasaremos los pesos usando el parámetro `s`. Dado que los pesos normalizados están entre 0 y 1, no serán visibles en el gráfico. Por lo tanto, haremos lo siguiente:
     * multiplicar los pesos por 2000 para aumentar la escala en el gráfico, y,
     * agregue 10 para compensar el valor mínimo (que tiene un peso de 0 y, por lo tanto, se escala con $\times 2000$).

In [ ]:
# Brazil
ax0 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='Brazil',
                    figsize=(14, 8),
                    alpha=0.5,  # transparencia
                    color='green',
                    s=norm_brazil * 2000 + 10,  # pasa el peso 
                    xlim=(1975, 2015)
                    )

# Argentina
ax1 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='Argentina',
                    alpha=0.5,
                    color="blue",
                    s=norm_argentina * 2000 + 10,
                    ax=ax0
                    )

ax0.set_ylabel('Número de Inmigrantes')
ax0.set_title('Inmigración desde Brazil y Argentina entre 1980 y 2013')
ax0.legend(['Brazil', 'Argentina'], loc='upper left', fontsize='x-large')

El tamaño de la burbuja corresponde a la magnitud de la población inmigrante para ese año, en comparación con los datos de 1980 - 2013. Cuanto más grande es la burbuja, más inmigrantes hay en ese año.

En el gráfico anterior, podemos ver un aumento correspondiente en la inmigración de Argentina durante la gran depresión de 1998 a 2002. También podemos observar un alza similar alrededor de 1985 a 1993. De hecho, Argentina había sufrido una gran depresión de 1974 a 1990, justo antes del inicio de la gran depresión de 1998 a 2002.

En una nota similar, Brasil sufrió el *Efecto Samba* donde el real brasileño (moneda) cayó casi un 35% en 1999. Existía el temor de una crisis financiera sudamericana ya que muchos países sudamericanos dependían en gran medida de las exportaciones industriales de Brasil. Posteriormente, el gobierno brasileño adoptó un programa de austeridad y la economía se recuperó lentamente a lo largo de los años, culminando en un aumento en 2010. Los datos de inmigración reflejan estos eventos.


**Pregunta**: Anteriormente se crearon diagramas de caja para comparar la inmigración de China e India a Canadá. Ahora, se pide crear diagramas de burbuja de la inmigración de China e India para visualizar cualquier diferencia con el tiempo desde 1980 hasta 2013. Se puede reutilizar el dataframe **df_can_t** que se definió y usó en el ejemplo anterior.


Paso 1: Normalizar los datos correspondiente a China e India.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo

# normalizar datos de China
# norm_china = ...
# normalizar datos de India
# norm_india = ...



Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  
    
# normalizar datos de China
norm_china = (df_can_t['China'] - df_can_t['China'].min()) / (df_can_t['China'].max() - df_can_t['China'].min())
# normalizar datos de India
norm_india = (df_can_t['India'] - df_can_t['India'].min()) / (df_can_t['India'].max() - df_can_t['India'].min())


-->

Paso 2: Crear los gráficos de burbuja.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo



Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:   
    
# China
ax0 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='China',
                    figsize=(14, 8),
                    alpha=0.5,               # transparencia
                    color='green',
                    s=norm_china * 2000 + 10,# pasa el peso 
                    xlim=(1975, 2015)
                    )

# India
ax1 = df_can_t.plot(kind='scatter',
                    x='Año',
                    y='India',
                    alpha=0.5,
                    color="blue",
                    s=norm_india * 2000 + 10,
                    ax = ax0
                    )

ax0.set_ylabel('Número de Inmigrantes')
ax0.set_title('Inmigración desde China e India entre 1980 y 2013')
ax0.legend(['China', 'India'], loc='upper left', fontsize='x-large')

-->